# Data prepration

In [1]:
import numpy as np
import datetime
import os
import sklearn.metrics
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from scipy.stats import chi
from scipy import stats

2022-08-04 11:53:25.148582: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-04 11:53:25.148612: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
############
# Functions
############

# Our 3d parameter space is the surface of a sphere centered at (1, 1, 1), with radius 1/sqrt(2)
# x, y coordinates are the x and y variances of our 2d Gaussian
# z coordinate determines the x-coordinate of the mean, mu = (z, 0)

def mean_gen(theta, phi):
    mu = 0.5 - (np.sqrt(2)/2) * np.cos(phi + np.pi / 4)
    return mu


def varx_gen(theta, phi):
    vx = 1.5 - (np.sqrt(2)/2) * np.cos(theta + np.pi / 4) * np.sin(phi + np.pi / 2)
    return vx


def vary_gen(theta, phi):
    vy = 1.5 - (np.sqrt(2)/2) * np.sin(theta + np.pi / 4) * np.sin(phi + np.pi / 2)
    return vy


# Generate n data
def spherical_data(n, thetas, phis, rand=1234):
    
    mx1, my1 = np.zeros(n), np.zeros(n)
    vx1, vy1 = np.ones(n), np.ones(n)
    
    mx2, my2 = mean_gen(thetas, phis), np.zeros(n)
    vx2, vy2 = varx_gen(thetas, phis), vary_gen(thetas, phis)
    
    x1, y1 = np.transpose(np.array([np.random.normal(mx1, vx1, size=n), 
                                    np.random.normal(my1, vy1, size=n), thetas, phis])), np.zeros(n)
    x2, y2 = np.transpose(np.array([np.random.normal(mx2, vx2, size=n), 
                                    np.random.normal(my2, vy2, size=n), thetas, phis])), np.ones(n)
    
    x, y = np.append(x1, x2, axis=0), np.append(y1, y2, axis=0)

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = rand)
    
    return x_train, x_test, y_train, y_test


def discrete_angles(n, m, s, rand = 1234):
    angles = np.random.uniform(0, s * np.pi, m)
    xk = np.arange(m)
    pk = (1 / m) * np.ones(int(m))
    discrete_distr = stats.rv_discrete(name='discrete_distr', values=(xk, pk), seed=rand)
    thetas = angles[discrete_distr.rvs(size=n)]
    return thetas

In [5]:
n = 300000


m_th = 40
m_ph = 40
thetas = discrete_angles(n, m_th, 2)
phis = discrete_angles(n, m_ph, 1)
name = ('discrete_model_mth%.0f_mph%.0f' %(m_th, m_ph))

# theta = 3
# phi = 3
# thetas = theta * np.ones(n)
# phis = phi * np.ones(n)
# name = ('discrete_model_th%.0f_ph%.0f' %(theta, phi))

x_train, x_test, y_train, y_test = spherical_data(n, thetas, phis)

print(x_train)

[[-0.736373    0.56009865  4.29622438  2.76084128]
 [ 0.51832278 -0.40341259  2.9093302   0.78697457]
 [ 0.80090189 -0.67188444  1.59190084  0.24153612]
 ...
 [ 1.34736101  0.09975717  3.58833442  2.12423857]
 [ 1.16147638 -0.47414315  3.33928396  1.47317794]
 [ 4.16035644 -0.31704087  1.54710856  1.6097847 ]]


# Training Interpolated Model

In [6]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
]);

loss_fn = tf.keras.losses.BinaryCrossentropy()
model.compile(
    optimizer='adam',
    loss=loss_fn,
    metrics=['accuracy']
)

batch_size = 1000

checkpoint_path = "tutorial/checkpoints/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    verbose=1,
    save_freq=10*batch_size
)

log_dir = "tutorial/logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

EPOCHS=20
model.fit(
    x_train, y_train, epochs=EPOCHS,
    batch_size=batch_size,
    callbacks=[cp_callback, tensorboard_callback]
);

model.save('3dmodels/' + name)

2022-08-04 11:54:08.564332: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-08-04 11:54:08.564509: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-08-04 11:54:08.564922: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


Epoch 1/20
 29/480 [>.............................] - ETA: 2s - loss: 0.6348 - accuracy: 0.6316

2022-08-04 11:54:09.031806: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-08-04 11:54:09.031838: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-08-04 11:54:09.051963: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-08-04 11:54:09.053019: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2022-08-04 11:54:09.065055: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: tutorial/logs/20220804-115408/train/plugins/profile/2022_08_04_11_54_09

2022-08-04 11:54:09.070296: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to tutorial/logs/20220804-115408/train/plugins/profile/2022_08_04_11_54_09/jupyter-matan.trace.json.gz
2022-08-04 11:54:09.072203: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: tutorial/logs/20220804-115

480/480 [==============================] - 2s 4ms/step - loss: 0.5839 - accuracy: 0.6828
Epoch 2/20
480/480 [==============================] - 2s 4ms/step - loss: 0.5744 - accuracy: 0.6906
Epoch 3/20
480/480 [==============================] - 2s 4ms/step - loss: 0.5726 - accuracy: 0.6915
Epoch 4/20
480/480 [==============================] - 2s 4ms/step - loss: 0.5720 - accuracy: 0.6918
Epoch 5/20
480/480 [==============================] - 2s 4ms/step - loss: 0.5712 - accuracy: 0.6930
Epoch 6/20
480/480 [==============================] - 2s 4ms/step - loss: 0.5706 - accuracy: 0.6932
Epoch 7/20
480/480 [==============================] - 2s 4ms/step - loss: 0.5702 - accuracy: 0.6935
Epoch 8/20
480/480 [==============================] - 2s 4ms/step - loss: 0.5700 - accuracy: 0.6938
Epoch 9/20
480/480 [==============================] - 2s 4ms/step - loss: 0.5696 - accuracy: 0.6943
Epoch 10/20
480/480 [==============================] - 2s 4ms/step - loss: 0.5693 - accuracy: 0.6945
Epoch 11/2

In [16]:
model = tf.keras.models.load_model('3dmodels/' + name)